In [ ]:
from sagemaker.chainer import Chainer
from sagemaker import get_execution_role
import sagemaker
role = get_execution_role()

bucket_name ="<bucketname>"

sagemaker_session = sagemaker.Session()

This launches the job on a device with no cpu.. hence slow ..

In [ ]:
chainer_estimator = Chainer("main_full_cycle.py",
                            role=role,
                        train_instance_type="ml.p3.8xlarge",
                        train_instance_count=1,
                        source_dir="custom_chainer",
                        hyperparameters = {'epoch': 10, 'batchsize': 50, 'gpu':0, 'traindata':'yelp_reviews.train.shuffled.csv', 'testdata':'yelp_reviews.test.shuffled.csv'
                                          ,'pretrained-embed':'glove.6B.300d.txt'})

chainer_estimator.fit({'train': 's3://{}/yelp_reviews.train.shuffled.csv'.format(bucket_name),
                   'test': 's3://{}/yelp_reviews.test.shuffled.csv'.format(bucket_name)
                      , 'PRETRAINEDEMBED' : 's3://{}/glove.6B.300d.txt'.format(bucket_name) })

In [ ]:
import boto3
from urllib.parse import urlparse

def rename_object(s3_source_url, s3_destination_url):      
    s = urlparse(s3_source_url)
    d = urlparse(s3_destination_url)
    s3 = boto3.resource('s3')
    s3.Object(d.netloc, d.path.lstrip('/')).copy_from(CopySource=s.netloc+s.path)

In [ ]:
chainer_training_job = chainer_estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=chainer_training_job)
model_data = desc['ModelArtifacts']['S3ModelArtifacts'] #.replace('model.tar.gz', 'output.tar.gz')
output_data = model_data.replace('model.tar.gz', 'output.tar.gz')

rename_object(output_data, model_data)
#retrieve_output_from_s3(output_data, 'output/sentiment')

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')